In [14]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [15]:
df = pd.read_csv('../data/weeklyRobloxComments.csv')

In [16]:
df.head()

,url,post date,post id,author,date,timestamp,score,comment,comment_id,parent_id
0,https://www.reddit.com/r/roblox/comments/11kiq...,2023-03-07,11kiq8l,JJthedog,2023-03-07,1.678154e+09,1,"New here, I play with my son who is about 5 a...",jb7reix,NaN
1,https://www.reddit.com/r/roblox/comments/11kiq...,2023-03-07,11kiq8l,Minute_Cantaloupe_78,2023-03-10,1.678461e+09,1,How can I recover my Niece’s account?\n\nMy ni...,jbol41h,NaN
2,https://www.reddit.com/r/roblox/comments/11kiq...,2023-03-07,11kiq8l,Muted_Ad_7704,2023-03-13,1.678715e+09,1,My wife set up a Roblox account for our son d...,jc1z21i,NaN
3,https://www.reddit.com/r/roblox/comments/11dr3...,2023-02-28,11dr3u4,garyfugazigary,2023-03-01,1.677643e+09,2,can anybody help with a mission my son is str...,jafznfe,NaN
4,https://www.reddit.com/r/roblox/comments/11dr3...,2023-02-28,11dr3u4,cryptogirl2002,2023-03-01,1.677663e+09,1,Using same account as daughter?\n\nHey so my d...,jagtc9s,NaN


In [17]:
df['comment']

0      New here,  I play with my son who is about 5 a...
1      How can I recover my Niece’s account?\n\nMy ni...
2       My wife set up a Roblox account for our son d...
3      can anybody  help with a mission my son is str...
4      Using same account as daughter?\n\nHey so my d...
                             ...                        
822    They're not allowed but it's just really hard ...
823    I restricted my son's account using the Roblox...
824    Roblox Account - Xbox Profile\n\nMy son has be...
825    i'm looking for a budget ipad for my daughter ...
826    did you set a usable email before the PIN was ...
Name: comment, Length: 827, dtype: object

In [18]:
df['comment'].str.count('game').sum()

733

In [19]:
text = " ".join(df['comment'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [20]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [21]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: ca nt with a frequency of 106


In [22]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                     term   rank
3539               ca nt  112.0
18966     roblox account   66.0
19285     roblox support   47.0
26509           year old   45.0
16757        play roblox   40.0
15901  parental controls   38.0
9132           gift card   35.0


In [23]:
df_ques = df['comment'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['comment'])

7      My son received a message from somebody not on...
15     https://preview.redd.it/cqkku0fizsia1.jpeg?wid...
18     \n\nMy son asked me to post here and ask where...
20     My grade-school son is into Roblox and I let h...
21     Parent of a new Roblox player kid, is there an...
                             ...                        
804    I just bought a new Chromebook for Christmas. ...
817    My son just started playing roblox on my Xbox ...
818    My daughter is trying to play Roblox on a 2011...
823    I restricted my son's account using the Roblox...
824    Roblox Account - Xbox Profile\n\nMy son has be...
Name: comment, Length: 196, dtype: object


In [24]:
df['text_processed'] = df['comment'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    new here  i play with my son who is about 5 an...
1    how can i recover my niece’s account\n\nmy nie...
2     my wife set up a roblox account for our son d...
3    can anybody  help with a mission my son is str...
4    using same account as daughter\n\nhey so my da...
Name: text_processed, dtype: object

In [25]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.comment.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['new', 'play', 'son', 'loves', 'roblox', 'everything', 'anyone', 'recommend', 'games', 'similar', 'laundry', 'simulator', 'smoothie', 'simulator', 'personally', 'get', 'little', 'bored', 'pizza', 'tycoon', 'mega', 'hotel', 'tycoon', 'also', 'need', 'stay', 'away', 'brookhaven', 'games', 'allow']


In [26]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
filename = 'WeeklyRobloxComments'
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('../results/'+filename)
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, '../results/'+ filename +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

c:\Users\mojea\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
